In [4]:
import gensim
import gensim.models.word2vec
from collections import defaultdict
import pickle
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process

In [35]:
model = gensim.models.Word2Vec.load_word2vec_format('/media/nikhil/Education/UCSC/PSL + OpenIE/word2vec/freebase-vectors-skipgram1000-en.bin.gz', binary=True)

In [14]:
#vocab = set(model.vocab.keys())

In [16]:
#pickle.dump(vocab,open('vocab-freebase-vectors-skipgram1000-en.pkl','wb'))

In [20]:
vocab = pickle.load(open('vocab-freebase-vectors-skipgram1000-en.pkl','rb'))
i = 5
for entity in vocab:
    i -= 1
    print(entity)
    if i == 0:
        break

/en/assi_abutbul
/en/cfb_goose_bay
/en/glasgow_hutchesons_aloysians_rfc
/en/oros
/en/jack_bruce


In [5]:
## Create dictionary to translate the id to the name

id2name = dict()

with open('names.txt') as f:
    for line in f:
        uniqid,name = line.strip('\n').split('\t')
        id2name[int(uniqid)] = name

In [6]:
id2name[181130]

'citycapitalofcountry'

In [7]:
uniq_rel = set()
valid_triples = defaultdict(set)
invalid_triples = defaultdict(set)

# Read the training relations file and separate the valid relations from the invalid relations
with open('label-train-uniq-raw-rel.db.TRAIN') as f:
    for line in f:
        
        e1id,e2id,relid,truth = line.strip('\n').split('\t')
        
        e1 = id2name[int(e1id)]
        e2 = id2name[int(e2id)]
        rel = id2name[int(relid)]
        
        uniq_rel.add(rel)
        
        if truth == '1':
            valid_triples[rel].add((e1,e2))
        elif truth == '0':
            invalid_triples[rel].add((e1,e2))
        else:
            print("strange truth value found")
            
pickle.dump(id2name, open('pickles/names.txt-id2name.pkl','wb'))
pickle.dump(valid_triples,open('pickles/label-train-uniq-raw-rel.db.TRAIN-valid_triples.pkl','wb'))
pickle.dump(invalid_triples,open('pickles/label-train-uniq-raw-rel.db.TRAIN-invalid_triples.pkl','wb'))

In [24]:
uniq_rel = set()
valid_triples = defaultdict(set)
invalid_triples = defaultdict(set)
candidate_triples = defaultdict(set)

filelist = ["label-train-uniq-raw-rel.db.TRAIN", "label-test-uniq-raw-rel.db.TRAIN", "NELL.08m.165.cesv.csv.CandRel_CBL.out", "NELL.08m.165.cesv.csv.CandRel_CPL.out", "NELL.08m.165.cesv.csv.CandRel_General.out", "NELL.08m.165.cesv.csv.CandRel.out", "NELL.08m.165.cesv.csv.CandRel_SEAL.out", "NELL.08m.165.cesv.csv.PattRel.out", "NELL.08m.165.esv.csv.PromRel_General.out", "seed.165.rel.uniq.out", "seed.165.rel.uniq_te.out", "testTargets.additional.Rel.out", "testTargets.additional.ValRel.out", "testTargets.shangpu.Rel.out", "testTargets.shangpu.ValRel.out", "trainTargets.Rel.out", "trainTargets.ValRel.out", "wlTargets.Rel.out"]

# Read ALL POSSIBLE RELATIONS FILES and separate the valid relations (Truth value 1) from 
# the invalid relations (Truth value 0) from the candidate relations (Truth value (0,1))

for file in filelist:
    with open(file) as f:
        for line in f:
            e1id,e2id,relid,truth = line.strip('\n').split('\t')

            e1 = int(e1id)
            e2 = int(e2id)
            rel = int(relid)
            uniq_rel.add(rel)
            
            if truth == '1':
                valid_triples[rel].add((e1,e2))
            elif truth == '0':
                invalid_triples[rel].add((e1,e2))
            else:
                candidate_triples[rel].add((e1,e2))


In [27]:
candidate_triples_count = 0
valid_triples_count = 0
invalid_triples_count = 0

for key in valid_triples.keys():
    valid_triples_count += len(valid_triples[key])

for key in invalid_triples.keys():
    invalid_triples_count += len(invalid_triples[key])

for key in candidate_triples.keys():
    candidate_triples_count += len(candidate_triples[key])
    
print(len(uniq_rel))
print(valid_triples_count)
print(invalid_triples_count)
print(candidate_triples_count)

275
72769
11701
194419


In [28]:
pickle.dump(uniq_rel, open('pickles/uniq_rel.pkl','wb'))
pickle.dump(valid_triples,open('pickles/valid_triples.pkl','wb'))
pickle.dump(invalid_triples,open('pickles/invalid_triples.pkl','wb'))
pickle.dump(candidate_triples,open('pickles/candidate_triples.pkl','wb'))

In [32]:
# Relation files MINUS CANDIDATE RELATIONS
# Counts the number of *true* relations that are usable for calculating the average relation vector

filelist = ["label-train-uniq-raw-rel.db.TRAIN",
            "label-test-uniq-raw-rel.db.TRAIN",
            "seed.165.rel.uniq.out",
            "seed.165.rel.uniq_te.out",
            "testTargets.additional.Rel.out", 
            "testTargets.additional.ValRel.out", 
            "testTargets.shangpu.Rel.out",
            "testTargets.shangpu.ValRel.out", 
            "trainTargets.Rel.out", 
            "trainTargets.ValRel.out",
            "wlTargets.Rel.out"]

cum_both_entities_in_word2vec = set()
cum_one_or_both_entities_not_in_word2vec = set()

for file in filelist:
    
    print("file:",file)
    
    both_entities_in_word2vec = set()
    one_or_both_entities_not_in_word2vec = set()
    
    with open(file) as f:
        
        for line in f:
            
            e1id,e2id,relid,truth = line.strip('\n').split('\t')
            
            e1 = id2name[int(e1id)]
            e2 = id2name[int(e2id)]
            
            if truth == '1':
                if '/en/'+e1 not in vocab or '/en/'+e2 not in vocab:
                    one_or_both_entities_not_in_word2vec.add((e1,e2))
                else:
                    both_entities_in_word2vec.add((e1,e2))
                    
    cum_both_entities_in_word2vec.update(both_entities_in_word2vec)
    cum_one_or_both_entities_not_in_word2vec.update(one_or_both_entities_not_in_word2vec)
                    
    print(len(both_entities_in_word2vec)," : both_entities_in_word2vec")
    print(len(one_or_both_entities_not_in_word2vec)," : one_or_both_entities_not_in_word2vec")
    pickle.dump(both_entities_in_word2vec,open('pickles/'+file+'-both-entities-in-w2v.pkl','wb'))
    pickle.dump(one_or_both_entities_not_in_word2vec,open('pickles/'+file+'-one-or-both-entities-not-in-w2v.pkl','wb'))
    print()

print("Cumulatively:")
print(len(cum_both_entities_in_word2vec)," : both_entities_in_word2vec")
print(len(cum_one_or_both_entities_not_in_word2vec)," : one_or_both_entities_not_in_word2vec")
pickle.dump(both_entities_in_word2vec,open('pickles/cum_both-entities-in-w2v.pkl','wb'))
pickle.dump(one_or_both_entities_not_in_word2vec,open('pickles/cum_one-or-both-entities-not-in-w2v.pkl','wb'))

file: label-train-uniq-raw-rel.db.TRAIN
1849  : both_entities_in_word2vec
2533  : one_or_both_entities_not_in_word2vec:

file: label-test-uniq-raw-rel.db.TRAIN
808  : both_entities_in_word2vec
1012  : one_or_both_entities_not_in_word2vec:

file: seed.165.rel.uniq.out
1209  : both_entities_in_word2vec
1276  : one_or_both_entities_not_in_word2vec:

file: seed.165.rel.uniq_te.out
1174  : both_entities_in_word2vec
1252  : one_or_both_entities_not_in_word2vec:

file: testTargets.additional.Rel.out
21  : both_entities_in_word2vec
39  : one_or_both_entities_not_in_word2vec:

file: testTargets.additional.ValRel.out
21  : both_entities_in_word2vec
39  : one_or_both_entities_not_in_word2vec:

file: testTargets.shangpu.Rel.out
1944  : both_entities_in_word2vec
1936  : one_or_both_entities_not_in_word2vec:

file: testTargets.shangpu.ValRel.out
1958  : both_entities_in_word2vec
1955  : one_or_both_entities_not_in_word2vec:

file: trainTargets.Rel.out
4330  : both_entities_in_word2vec
5848  : one_or

In [34]:
'''
Get the true relations in the form
 {
    relation1: [(e11,e12),(e13,e14)...],
    relation2: [(e21,e22),(e23,e24)...]
 }
'''

preproc_for_avg_rel_vec = defaultdict(set)

filelist = ["label-train-uniq-raw-rel.db.TRAIN", "label-test-uniq-raw-rel.db.TRAIN", "seed.165.rel.uniq.out", "seed.165.rel.uniq_te.out", "testTargets.additional.Rel.out", "testTargets.additional.ValRel.out", "testTargets.shangpu.Rel.out", "testTargets.shangpu.ValRel.out", "trainTargets.Rel.out", "trainTargets.ValRel.out", "wlTargets.Rel.out"]

for file in filelist:
    
    with open(file) as f:
        
        for line in f:
            
            e1id,e2id,relid,truth = line.strip('\n').split('\t')
            
            e1 = id2name[int(e1id)]
            e2 = id2name[int(e2id)]
            
            if truth == '1' and '/en/'+e1 in vocab and '/en/'+e2 in vocab:
                preproc_for_avg_rel_vec[int(relid)].add(('/en/'+e1,'/en/'+e2))
                
len(preproc_for_avg_rel_vec)

157

In [37]:
'''
Calculate average relation vector for the 157 unique relations
'''

avg_rel_vectors = dict()

for rel,elist in preproc_for_avg_rel_vec.items():
    n = len(elist)
    vector_sum = 0
    for e1,e2 in elist:
        vector_sum += model[e1] - model[e2]
    avg_rel_vectors[rel] = vector_sum / n

In [40]:
print(len(avg_rel_vectors))
pickle.dump(avg_rel_vectors,open('pickles/avg_rel_vectors-157-relations.pkl','wb'))

157


In [45]:
for e in not_in_model_list:
    print(e, ":", process.extract(e, candidates, limit=2))
#This proceeds roughly at the rate of 1 entity per 10 minutes, and doesn't even return satisfying results.
#Killed it after about 25 hours

seattle_redhawks : [('/en/seattle_seahawks', 80), ('/en/seattle_seahawks_ring_of_honor', 79)]
bobby_mosebach : [('/en/bach', 75), ('/en/bobby_hosea', 71)]
ohio_university_bobcats : [('/en/ohio_university', 73), ('/en/ohio_university_press', 72)]
virginia_tech : [('/en/virginia_tech_hokies', 90), ('/en/virginia_tech_massacre', 90)]
o__j__simpson : [('/en/o_j_simpson', 81), ('/en/o_j_simpson_murder_case', 76)]
bears : [('/en/birmingham_bears', 90), ('/en/doosan_bears', 90)]
uc_santa_barbara_gauchos : [('/en/santa_barbara', 73), ('/en/ana_barbara', 71)]
minnesota_duluth_bulldogs : [('/en/duluth_bulldogs', 74), ('/en/west_duluth', 71)]
pats : [('/en/patsy_rowe', 90), ('/en/patsy_calton', 90)]
adobe_website : [('/en/corporate_website', 75), ('/en/website', 70)]
america_west_arena : [('/en/en', 86), ('/en/american_west', 78)]
dodgers_stadium : [('/en/dodger_stadium', 88), ('/en/rogers_stadium', 81)]
ucla : [('/en/roussel_uclaf', 90), ('/en/ucla_taser_incident', 90)]
scott_baker : [('/en/scot

KeyboardInterrupt: 

In [11]:
i = 0
for key in model.vocab.keys():
    print(key)
    if i == 5:
        break
    i += 1

/m/0ch0_f6
/en/shane_howarth
/en/fellsmere
/en/russian_first_division
/en/albert_bridge_brisbane
/en/criminal_defense_lawyer


In [ ]:
all_vectors = np.zeros((1000,1))
#print(all_vectors)
#print(model['/en/shane_howarth'])

In [50]:
x.shape

(1000,)

In [53]:
x[0][0],x[1],x[2]

IndexError: invalid index to scalar variable.